<div style="font-family:verdana">

<h3> <b>🏘 Sistema de diàleg basat en regles 🏘</b> </h3>
<p style="font-size:16px;"> Pràctica 1 </p>
<p style="font-size:14px;"> Paula Justo i Júlia Pedrol </p>
<p style="font-size:14px;"> 3r GIA - TVD </p>

</div>

<p>   <b>Objectiu</b>: en aquesta pràctica aprendrem a crear un assistent senzill de compra d'habitatges. El sistema haurà d'identificar i mostrar les cases que coincideixen amb les preferències de l'usuari. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

<p> <b>Contingut</b>: en primer lloc, començarem amb un exemple senzill en què el sistema llançarà unes preguntes i respostes predefinides i l'usuari haurà d'escollir. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

<b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent.

---

<h2> Índex </h2>


1. [Fitxer JSON](#section-one)
  * [Exercici 1](#ex-one)
2. [Sistema de diàleg senzill](#section-two)
  * [Exercici 2](#ex-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Millorem el sistema de diàleg](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)
---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import json
import sys

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


<h1><a name="section-one"> 1. Fitxer JSON </a></h1>

En primer lloc, carregarem el fitxer JSON per analitzar-ne l'estructura.

In [ ]:
from google.colab import files
import json

uploaded = files.upload()

with open('house_data.json', 'r') as f:
  data = json.load(f)

print('Start message: ', data['start_message'])
print('End message: ', data['end_message'])
print('Questions: ', data['questions'])
print('House: ', data['houses'])

Saving house_data.json to house_data (2).json
Start message:  Welcome to the House Buying Assistant!
End message:  Thank you for using the House Buying Assistant. Goodbye!
Questions:  [{'question': 'How many bedrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 5 bedrooms): ', 'answer_key': 'bedrooms'}, {'question': 'How many bathrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 3 bathrooms): ', 'answer_key': 'bathrooms'}, {'question': 'What is your budget for the house?', 'type': 'numerical', 'prompt': 'Enter your choice (1k - 999k euros):', 'answer_key': 'price'}, {'question': 'How many square meters do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (30 - 200 meters)', 'answer_key': 'square_meters'}, {'question': 'Which city or neighborhood would you prefer?', 'type': 'multichoice', 'prompt': 'Enter your choice', 'answer_key': 'location'}]
House:  [{'id': 1, 'type': 'rent', 'bedrooms': '3', 'bathrooms': '2', 'price': '

Podeu veure com al fitxer JSON podem trobar el missatge de benvinguda i el missatge de comiat. Les diferents preguntes predefinides que el sistema demanarà a l'usuari. I les cases que l'agència té disponibles per a oferir a l'usuari.

Si no acabeu d'entendre el format, podeu copiar el contingut del fitxer JSON [aquí](https://jsonviewer.stack.hu/) per visualitzar millor l'estructura.

---



 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>


En aquest primer exercici us demanem que afegiu una nova casa manualment al fitxer JSON amb l'id 26. La resta de camps els podeu emplenar com vulgueu.

Si ho heu fet correctament podreu veure al següent *print* les dades que heu introduït.

In [ ]:
ids = {h["id"] for h in data["houses"]}
if 26 in ids:
    print("Ja existeix una casa amb id=26.")
else:
    new_house = {
        "id": 26,
        "type": "sale",
        "bedrooms": "2",
        "bathrooms": "1",
        "price": "90k",
        "square_meters": "70",
        "floor": "1",
        "elevator": "Yes",
        "commercial_use": "No",
        "terrace": "Yes",
        "location": "Badalona"
    }

    data["houses"].append(new_house)

    with open('house_data.json', 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

Ja existeix una casa amb id=26.


In [ ]:
id = 26
house = data['houses'][id-1]

print(f"House ID: {house['id']}")
print(f"Bedrooms: {house['bedrooms']}")
print(f"Bathrooms: {house['bathrooms']}")
print(f"Price: {house['price']}")
print(f"Square Meters: {house['square_meters']}")
print(f"Location: {house['location']}")

House ID: 26
Bedrooms: 2
Bathrooms: 1
Price: 90k
Square Meters: 70
Location: Badalona


---

<h1><a name="section-two"> 2. Sistema de diàleg senzill </a></h1>


En aquesta primera part crearem un sistema senzill que mostrarà les preguntes predefinides i, a continuació l'usuari haurà d'introduir la resposta. Tractarem de manera diferent les preguntes en què la resposta siga un número i les preguntes d'opció múltiple.

Intenteu entendre el funcionament, ja que treballarem sobre aquest codi.

In [ ]:
def print_question(prompt, possible_options = []):
    """
    Imprimeix la pregunta i les opcions possibles si hi ha alguna definida.
    """
    print(prompt)
    if not len(possible_options) == 0:
        print("Options:", ", ".join(possible_options))

def initialize_available_options(house_data, available_options):
    """
    Inicialitza un diccionari amb les opcions disponibles a partir de les cases en house_data.
    """
    for house in house_data['houses']:
        for key, value in house.items():
            available_options.setdefault(key, set()).add(value)

def preprocess_answer(answer):
    """
    Tokenitza la resposta per separar-la en paraules o unitats significatives.
    """
    answer = nltk.word_tokenize(answer)
    return answer

def get_numerical_value(tok_answer):
    """
    Retorna el primer valor numèric trobat en la resposta tokenitzada.
    """
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric():
            return token
    return ''

#Exemple de frase tokenitzada
print(preprocess_answer('Vull un pis de 35k €'))

['Vull', 'un', 'pis', 'de', '35k', '€']


In [ ]:
def process_numerical_question(question):
    """
    Imprimeix la pregunta i processa una resposta numèrica fins a obtenir un valor vàlid.
    """
    print_question(question['question'])
    while True:
      answer = input(question['prompt'])
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)
      if not value == '':
        return value

def process_multichoice_question(question, options):
    """
    Imprimeix la pregunta i processa una resposta d'elecció múltiple fins a obtenir una opció vàlida.
    """
    print_question(question['question'], options)
    while True:
      answer = input(question['prompt'])
      if answer in options:
        return answer

user_preferences, available_options = {}, {}
initialize_available_options(data, available_options)

for question in data['questions']:
  answer_key = question['answer_key']
  possible_options = list(available_options.get(answer_key))

  if question['type'] == 'numerical':
    answer = process_numerical_question(question)
  else:
    answer = process_multichoice_question(question, possible_options)

  user_preferences[answer_key] = answer


How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 3
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 2
What is your budget for the house?
Enter your choice (1k - 999k euros):1200
How many square meters do you need?
Enter your choice (30 - 200 meters)60 
Which city or neighborhood would you prefer?
Options: Santa Coloma de Gramenet, Esplugues de Llobregat, L'Hospitalet de Llobregat, Badalona, Barcelona
Enter your choicesanta Coloma de gram 
Enter your choiceSanta Coloma de Gramenet




---


 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏ </a></h1>

A continuació millorarem un poc el sistema. Us proposem que afegiu la frase de benvinguda que tenim guardada al fitxer JSON. És a dir, abans de mostrar la primera pregunta el sistema ens donarà la benvinguda.

In [ ]:
import time

def benvinguda(data, delay: float = 1.5):
    """
    Imprimeix el missatge de benvinguda i fa una pausa abans de continuar.
    """
    print(data['start_message'])
    time.sleep(delay)

def acomiadament(data, delay: float = 1.0):
    """
    Fa una pausa i imprimeix el missatge d'acomiadament.
    """
    time.sleep(delay)
    print(data['end_message'])


In [ ]:
def questionari(data):
    """
    Processa les preguntes d'un qüestionari, recollint les respostes de l'usuari i retornant-les en un diccionari.
    """
    user_preferences, available_options = {}, {}
    initialize_available_options(data, available_options)

    for question in data['questions']:
        answer_key = question['answer_key']
        possible_options = list(available_options.get(answer_key, []))

        if question['type'] == 'numerical':
            answer = process_numerical_question(question)
        else:
            answer = process_multichoice_question(question, possible_options)

        user_preferences[answer_key] = answer

    return user_preferences


In [ ]:
benvinguda(data)
user_preferences = questionari(data)
acomiadament(data)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): quit
Enter your choice (1 - 5 bedrooms): 5
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 2
What is your budget for the house?
Enter your choice (1k - 999k euros):1200
How many square meters do you need?
Enter your choice (30 - 200 meters)60 
Which city or neighborhood would you prefer?
Options: Santa Coloma de Gramenet, Esplugues de Llobregat, L'Hospitalet de Llobregat, Badalona, Barcelona
Enter your choiceSanta Coloma de Gramenet
Thank you for using the House Buying Assistant. Goodbye!


---


 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏ </a></h1>

En aquest exercici us demanem que permeteu a l'usuari abandonar el programa quan ho desitge. Per exemple, si l'usuari escriu `quit`, que el sistema s'acomiade amb l'oració de comiat que trobareu al JSON i finalitze el programa.


🙃 `sys.exit()` genera una excepció SystemExit.

In [ ]:
import sys
import time

def process_numerical_question(question):
    """
    Imprimeix la pregunta i processa una resposta numèrica fins a obtenir un valor vàlid o una comanda 'quit' per sortir.
    """
    print_question(question['question'])
    while True:
        answer = input(question['prompt'])

        if answer.lower().strip() == "quit":
            raise SystemExit

        tok_answer = preprocess_answer(answer)
        value = get_numerical_value(tok_answer)
        if value != '':
            return value

def process_multichoice_question(question, options):
    """
    Imprimeix la pregunta i processa una resposta d'elecció múltiple fins a obtenir una opció vàlida o una comanda 'quit' per sortir.
    """
    print_question(question['question'], options)
    while True:
        answer = input(question['prompt'])

        if answer.lower().strip() == "quit":
            raise SystemExit

        if answer in options:
            return answer


In [ ]:
try:
    benvinguda(data)
    user_preferences = questionari(data)
    acomiadament(data)

except SystemExit:
    acomiadament(data)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): Quit 
Thank you for using the House Buying Assistant. Goodbye!


**Funciona també si escriviu `Quit`?**

Sí, ja que hem fet que el propi programa un cop rebi la resposta la passi tota a minuscules per tal d'evitar que el sistema falli i així pugui processar corrrectament la resposta.

---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏ </a></h1>

Fins ara tenim a `user_preferences` les opcions que l'usuari ha escollit.







In [ ]:
user_preferences

{'bedrooms': '5',
 'bathrooms': '2',
 'price': '1200',
 'square_meters': '60',
 'location': 'Santa Coloma de Gramenet'}

Per tant, ara haurem de revisar si l'agència disposa d'alguna casa disponible amb aquestes característiques. Per això us proposem completar la funció `find_suitable_houses`. Aquesta funció tornarà una llista amb tants diccionaris com cases disponibles. Per exemple si segons les preferències de l'usuari només està disponible la casa amb id 3, `suitable_house` serà:


```
[{'id': 3, 'bedrooms': '2', 'bathrooms': '1', 'price': '160k', 'square_meters': '80', 'location': 'Santa Coloma de Gramenet'}]
```

In [ ]:
def _to_int(s):
    """
    Converteix una cadena a un enter, eliminant qualsevol caràcter no numèric.
    """
    if s is None:
        return None
    digits = re.sub(r'[^0-9]', '', str(s))
    return int(digits) if digits else None

def _to_euros(s):
    """
    Converteix una cadena a un valor numèric en euros, considerant formats amb '€' o 'euros' i 'k' per milers.
    """
    if s is None:
        return None
    txt = str(s).lower().replace('€', '').replace('euros', '').strip()
    m = re.match(r'^\s*([0-9]+)\s*([k]?)\s*$', txt)
    if m:
        n = int(m.group(1))
        return n * 1000 if m.group(2) == 'k' else n
    digits = re.sub(r'[^0-9]', '', txt)
    return int(digits) if digits else None

def find_suitable_houses(data, user_preferences):
    """
    Troba les cases que compleixen amb els criteris especificats per l'usuari, com nombre d'habitacions, preu, ubicació, etc.
    """
    houses = data.get('houses', [])
    if not houses:
        return []

    min_bed = _to_int(user_preferences.get('bedrooms'))
    min_bath = _to_int(user_preferences.get('bathrooms'))
    min_sqm = _to_int(user_preferences.get('square_meters'))
    max_price = _to_euros(user_preferences.get('price'))
    wanted_loc = user_preferences.get('location')
    wanted_type = user_preferences.get('type')  # de cara a l’Ex.6

    suitable = []
    for h in houses:
        h_bed   = _to_int(h.get('bedrooms'))
        h_bath  = _to_int(h.get('bathrooms'))
        h_sqm   = _to_int(h.get('square_meters'))
        h_price = _to_euros(h.get('price'))
        h_loc   = (h.get('location') or '').strip().lower()
        h_type  = (h.get('type') or '').strip().lower()

        if min_bed  is not None and (h_bed  is None or h_bed  < min_bed):   continue
        if min_bath is not None and (h_bath is None or h_bath < min_bath):  continue
        if min_sqm  is not None and (h_sqm  is None or h_sqm  < min_sqm):   continue
        if max_price is not None and (h_price is None or h_price > max_price): continue

        if wanted_loc:
            if h_loc != wanted_loc.strip().lower():
                continue
        if wanted_type:
            if h_type != wanted_type.strip().lower():
                continue

        suitable.append(h)

    suitable.sort(key=lambda x: (_to_euros(x.get('price'))
                                 if _to_euros(x.get('price')) is not None else float('inf')))
    return suitable


---



Una vegada tingueu la funció `find_suitable_houses` programada, podreu executar el següent *script* que us mostrarà, si n'hi ha, la casa o cases disponibles amb les característiques que heu triat.

In [ ]:
import re

def print_suitable_houses(suitable_houses):
    """
    Imprimeix la llista de cases que compleixen amb les preferències de l'usuari o un missatge indicant que no s'han trobat resultats.
    """
    if suitable_houses:
        print("\nBased on your preferences, these are the properties that best suit you.\n")
        for house in suitable_houses:
            print(f"House ID: {house.get('id')}")
            print(f"Type: {house.get('type')}")
            print("Bedrooms:", house.get('bedrooms'))
            print("Bathrooms:", house.get('bathrooms'))
            print("Price:", house.get('price'))
            print("Square Meters:", house.get('square_meters'))
            print("Location:", house.get('location'))
            print()
    else:
        print("\nSorry! I haven't found any properties that match your preferences.\n")


suitable_houses = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable_houses)



Sorry! I haven't found any properties that match your preferences.



<h1><a name="section-three"> 3. Millorem el sistema de diàleg </a></h1>


---


 <h1><a name="ex-five"><center> ✏ Exercici 5 ✏ </a></h1>

Tal com haureu comprovat sempre esteu obligats a triar una opció. En aquest últim exercici proposem que afegiu l'opció de no triar-ne cap. És a dir, si ens és igual el nombre d'habitacions, podrem posar per exemple `any` i el sistema ens farà la següent pregunta.

Hem provat de fer això en exercicis anterirs i hem pogut comprovar que els servidor no actua de la mateixa manera, ja que si tu li prorciones com a repsosta alguna de les preguntres `any`, com que no enten aquesta resposta, et torna a repetir la mateixa pregunta, així també ens assegurem que el comportament del nou sistema és coherent amb el que demana l'exercici.

In [ ]:
import unicodedata

SKIP_WORDS = {
    "any", "qualsevol", "igual", "cap",
    "sense preferencia", "sense preferència",
    "none", "no", "skip"
}

def _normalize(text: str) -> str:
    """
    Normalitza el text, convertint-lo a minúscules i eliminant els accents.
    """
    text = text.lower().strip()
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    return text

def _is_skip(ans: str) -> bool:
    """
    Comprova si la resposta és una de les paraules que s'han de saltar (com 'no' o 'skip').
    """
    if ans is None:
        return False
    normalized = _normalize(ans)
    return normalized in {_normalize(w) for w in SKIP_WORDS}

def process_numerical_question(question):
    """
    Imprimeix la pregunta i processa una resposta numèrica fins a obtenir un valor vàlid o una comanda 'quit' per sortir o saltar la pregunta.
    """
    print_question(question['question'])
    while True:
        answer = input(question['prompt'])

        if answer.lower().strip() == "quit":
            raise SystemExit

        if _is_skip(answer):
            return None

        tok_answer = preprocess_answer(answer)
        value = get_numerical_value(tok_answer)
        if value != '':
            return value

def process_multichoice_question(question, options):
    """
    Imprimeix la pregunta i processa una resposta d'elecció múltiple fins a obtenir una opció vàlida o una comanda 'quit' per sortir o saltar la pregunta.
    """
    print_question(question['question'], options)
    while True:
        answer = input(question['prompt'])

        if answer.lower().strip() == "quit":
            raise SystemExit

        if _is_skip(answer):
            return None

        if answer in options:
            return answer

def questionari(data):
    """
    Processa les preguntes d'un qüestionari, recollint les respostes de l'usuari i retornant-les en un diccionari. Les respostes poden ser saltades.
    """
    user_preferences, available_options = {}, {}
    initialize_available_options(data, available_options)

    for question in data['questions']:
        answer_key = question['answer_key']
        possible_options = list(available_options.get(answer_key, []))

        if question['type'] == 'numerical':
            answer = process_numerical_question(question)
        else:
            answer = process_multichoice_question(question, possible_options)

        if answer is not None:
            user_preferences[answer_key] = answer

    return user_preferences


In [ ]:
benvinguda(data)
user_preferences = questionari(data)
suitable = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable)
acomiadament(data)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): any 
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): any 
What is your budget for the house?
Enter your choice (1k - 999k euros):any 
How many square meters do you need?
Enter your choice (30 - 200 meters)any 
Which city or neighborhood would you prefer?
Options: Santa Coloma de Gramenet, Esplugues de Llobregat, L'Hospitalet de Llobregat, Badalona, Barcelona
Enter your choiceny 
Enter your choiceany 

Based on your preferences, these are the properties that best suit you.

House ID: 17
Type: rent
Bedrooms: 1
Bathrooms: 1
Price: 450
Square Meters: 45
Location: Barcelona

House ID: 19
Type: rent
Bedrooms: 3
Bathrooms: 1
Price: 600
Square Meters: 75
Location: Barcelona

House ID: 5
Type: rent
Bedrooms: 1
Bathrooms: 1
Price: 650
Square Meters: 55
Location: Santa Coloma de Gramenet

House ID: 9
Type: rent
Bedrooms: 1
Bathrooms: 1
Price: 700
Square Meters: 55
Locatio

---



 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Amplieu el vostre xatbot perquè tinga en compte la següent informació sobre:

1. **Tipus d'habitatge:** Pregunteu a l'usuari si vol una casa per a compra (tipus *"sale"* a les dades) o per llogar (tipus *"rent"*). Al final de l'execució, el sistema haurá de mostrar habitatges amb el tipus seleccionat.

2. **Ingressos:** S'aconsella no dedicar més del 35% dels ingressos a l'habitatge. En cas d'escollir lloguer, preguntar a l'usuari els ingressos mensuals de la seua llar, i només mostrar aquells que estiguen per sota del 35%.

3. **Planta:** Preguntar als usuaris a quina planta volen viure. Al final de l'execució, només s'oferiràn habitatges que estiguen en aquella planta o superior.

4. **Terrassa:** Preguntar a l'usuari si vol un habitatge amb terrassa o no. En cas de voler terrassa, només s'oferiran habitatges amb terrassa. En cas contrari, oferir habitatges amb terrassa o sense.

5. **Ascensor:** Preguntar a l'usuari si vol un habitatge amb ascensor o no. En cas de voler ascensor, només s'oferiran habitatges amb ascensor. En cas contrari, oferir habitatges amb ascensor o sense.

6. **Us comercial:** Preguntar a l'usuari si vol fer servir l'habitatge com a negoci. En cas afirmatiu, només oferir habitatges amb la propietat *"commercial use"*

---





In [ ]:
import unicodedata
import re

def _normalize(text: str) -> str:
    """
    Normalitza el text, eliminant espais extra i signes diacrítics (accents).
    """
    if text is None:
        return ""
    text = text.strip().lower()
    text = "".join(
        c for c in unicodedata.normalize("NFD", text)
        if unicodedata.category(c) != "Mn"
    )
    return re.sub(r"\s+", " ", text)

def _to_bool(s):
    """
    Converteix una cadena en un valor booleà (True/False) basat en paraules clau com 'sí', 'no', 'true', 'false'.
    """
    if s is None:
        return None
    t = _normalize(str(s))
    TRUE = {"yes","si","sí","s","y","true","1","ok","vale","clar","correcte","sure"}
    FALSE = {"no","false","0","n","never","mai"}
    if t in TRUE:
        return True
    if t in FALSE:
        return False
    return None

def ask_type_or_skip():
    """
    Pregunta a l'usuari si vol comprar o llogar i permet saltar la pregunta amb 'any'.
    """
    while True:
        ans = input("Vols comprar o llogar? (sale/rent) [o escriu 'any' per saltar]: ").strip()
        if _normalize(ans) == "quit":
            raise SystemExit
        if _is_skip(ans):
            return None
        ansL = _normalize(ans)
        if ansL in {"sale","rent"}:
            return ansL
        print("Introdueix 'sale', 'rent' o 'any' per no filtrar.")

def ask_income_if_rent_or_skip(selected_type):
    """
    Si l'usuari ha seleccionat 'rent', pregunta els ingressos mensuals de la llar i permet saltar la pregunta amb 'any'.
    """
    if selected_type != "rent":
        return None
    while True:
        ans = input("Ingressos mensuals de la llar (€). Pots escriure 'any' per saltar: ").strip()
        if _normalize(ans) == "quit":
            raise SystemExit
        if _is_skip(ans):
            return None
        tokens = re.findall(r"\d+", ans)
        val = tokens[0] if tokens else ans
        euros = _to_euros(val)
        if euros is not None:
            return euros
        print("Introdueix una xifra com '2500' o escriu 'any' per passar.")

def ask_min_floor_or_skip():
    """
    Pregunta la planta mínima on l'usuari vol viure i permet saltar la pregunta amb 'any'.
    """
    while True:
        ans = input("Planta mínima on vols viure? (0 = PB) [o 'any' per saltar]: ").strip()
        if _normalize(ans) == "quit":
            raise SystemExit
        if _is_skip(ans):
            return None
        n = _to_int(ans)
        if n is not None:
            return n
        print("Introdueix un número enter (p.ex. 2) o 'any'.")

def ask_yesno_or_skip(prompt):
    """
    Pregunta una resposta de tipus 'sí' o 'no' i permet saltar la pregunta amb 'any'.
    """
    while True:
        ans = input(prompt + " (sí/no o 'any' per saltar): ").strip()
        if _normalize(ans) == "quit":
            raise SystemExit
        if _is_skip(ans):
            return None
        b = _to_bool(ans)
        if b is not None:
            return b
        print("Respon 'sí', 'no' o 'any' per passar.")

def questionari_ex6(data):
    """
    Recull les preferències de l'usuari a partir d'un qüestionari i altres preguntes opcionals, com el tipus de propietat i característiques desitjades.
    """
    user_preferences, available_options = {}, {}
    initialize_available_options(data, available_options)

    for question in data['questions']:
        answer_key = question['answer_key']
        possible_options = list(available_options.get(answer_key, []))

        if question['type'] == 'numerical':
            ans = process_numerical_question(question)
        else:
            ans = process_multichoice_question(question, possible_options)

        if ans is not None:
            user_preferences[answer_key] = ans

    selected_type = ask_type_or_skip()
    if selected_type is not None:
        user_preferences['type'] = selected_type

    income = ask_income_if_rent_or_skip(selected_type)
    if income is not None:
        user_preferences['income_monthly'] = income

    min_floor = ask_min_floor_or_skip()
    if min_floor is not None:
        user_preferences['min_floor'] = min_floor

    want_terrace = ask_yesno_or_skip("Vols terrassa?")
    if want_terrace is not None:
        user_preferences['want_terrace'] = want_terrace

    want_elevator = ask_yesno_or_skip("Vols ascensor?")
    if want_elevator is not None:
        user_preferences['want_elevator'] = want_elevator

    want_commercial = ask_yesno_or_skip("Faràs ús comercial (negoci)?")
    if want_commercial is not None:
        user_preferences['want_commercial_use'] = want_commercial

    return user_preferences


In [ ]:
def find_suitable_houses(data, user_preferences):
    """
    Troba les cases que compleixen amb les preferències de l'usuari, com nombre d'habitacions, preu, ubicació, tipus, ingressos, planta mínima, terrassa, ascensor i ús comercial.
    """
    houses = data.get('houses', [])
    if not houses: return []

    min_bed = _to_int(user_preferences.get('bedrooms'))
    min_bath = _to_int(user_preferences.get('bathrooms'))
    min_sqm = _to_int(user_preferences.get('square_meters'))
    max_price = _to_euros(user_preferences.get('price'))
    wanted_loc = (user_preferences.get('location') or None)
    wanted_type = (user_preferences.get('type') or None)

    income = user_preferences.get('income_monthly')
    min_floor = user_preferences.get('min_floor')
    want_terrace = user_preferences.get('want_terrace')
    want_elevator = user_preferences.get('want_elevator')
    want_commercial = user_preferences.get('want_commercial_use')

    out = []
    for h in houses:
        h_bed   = _to_int(h.get('bedrooms'))
        h_bath  = _to_int(h.get('bathrooms'))
        h_sqm   = _to_int(h.get('square_meters'))
        h_price = _to_euros(h.get('price'))
        h_loc   = (h.get('location') or '').strip().lower()
        h_type  = (h.get('type') or '').strip().lower()
        h_floor = _to_int(h.get('floor'))
        h_terr  = (h.get('terrace') or '').strip().lower() in {"yes","sí","si","true","1"}
        h_elev  = (h.get('elevator') or '').strip().lower() in {"yes","sí","si","true","1"}
        h_comm  = (h.get('commercial_use') or '').strip().lower() in {"yes","sí","si","true","1"}

        if min_bed  is not None and (h_bed  is None or h_bed  < min_bed): continue
        if min_bath is not None and (h_bath is None or h_bath < min_bath): continue
        if min_sqm  is not None and (h_sqm  is None or h_sqm  < min_sqm): continue
        if max_price is not None and (h_price is None or h_price > max_price): continue
        if wanted_loc and (h_loc != wanted_loc.strip().lower()): continue
        if wanted_type and (h_type != wanted_type.strip().lower()): continue

        if wanted_type == "rent" and income is not None:
            max_affordable = 0.35 * float(income)
            if h_price is None or h_price > max_affordable:
                continue

        if min_floor is not None and (h_floor is None or h_floor < int(min_floor)):
            continue

        if want_terrace is True and not h_terr:
            continue

        if want_elevator is True and not h_elev:
            continue

        if want_commercial is True and not h_comm:
            continue

        out.append(h)

    out.sort(key=lambda x: (_to_euros(x.get('price'))
                            if _to_euros(x.get('price')) is not None else float('inf')))
    return out


In [ ]:
def print_suitable_houses(suitable_houses):
    """
    Imprimeix la llista de cases que compleixen amb les preferències de l'usuari, mostrant les seves propietats com ID, tipus,
    nombre d'habitacions, banys, preu, m², planta, terrassa, ascensor, ús comercial i ubicació.
    """
    if suitable_houses:
        print("\nBased on your preferences, these are the properties that best suit you\n")
        for house in suitable_houses:
            print(f"House ID: {house.get('id')}")
            print(f"Type: {house.get('type')}")
            print("Bedrooms:", house.get('bedrooms'))
            print("Bathrooms:", house.get('bathrooms'))
            print("Price:", house.get('price'))
            print("Square Meters:", house.get('square_meters'))
            print("Floor:", house.get('floor'))
            print("Terrace:", house.get('terrace'))
            print("Elevator:", house.get('elevator'))
            print("Commercial use:", house.get('commercial_use'))
            print("Location:", house.get('location'))
            print()
    else:
        print("\nSorry! I haven't found any properties that match your preferences\n")


In [ ]:
try:
    benvinguda(data)
    user_preferences = questionari_ex6(data)
    suitable = find_suitable_houses(data, user_preferences)
    print_suitable_houses(suitable)
    acomiadament(data)
except SystemExit:
    acomiadament(data)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 7
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): quit
Thank you for using the House Buying Assistant. Goodbye!


 <h1><a name="ex-seven"><center> ✏ Exercici 7 ✏ </a></h1>

Ara ja teniu un sistema funcional. Aplicant el mètode vist a classe (Exemples, Camins, Prototipat i Proves) milloreu aquest sistema per donar una millor experiència als usuaris.

**Millora 1: Humanització del sistema de diàleg**

En aquesta primera millora, hem buscat fer que el nostre sistema de diàleg sigui més proper i natural, evitant respostes massa mecàniques o repetitives que podrien donar la sensació de sistema automatitzat.

L'objectiu és assolir una experiència que es senti més com una conversa amb una persona real. Per tal d'aconseguir-ho, hem definit diversos conjunts de frases en català que el sistema utilitza de manera aleatòria en funció del moment de la conversa, aconseguint així que el sistema sigui més variat i menys previsible. Aquests conjunts de frases són:
- **SALUTACIONS:** Aquest conjunt de frases es fa servir per donar la benvinguda a l'usuari de manera càlida i variada.
- **CONFIRMACIONS:** Aquest conjunt es fa servir per respondre després que l'usuari proporcioni una informació, reafirmant que s’ha entès correctament i que la resposta ha estat anotada.
- **REPETIR_PREGUNTA:** Aquest conjunt de frases s’utilitza quan l'usuari introdueix una resposta que no és vàlida, guiant-lo de manera clara i amable. Això evita respostes rígides i ajuda l'usuari a comprendre millor què se li demana.
- **TRANSICIONS:** Aquest conjunt ajuda a connectar una pregunta amb la següent, de manera que la conversa flueixi de manera natural i sense interrupcions.
- **COMIATS:** A l'hora de finalitzar la interacció amb l'usuari, utilitzem aquest conjunt de frases amables per assegurar-nos que l'usuari té una bona experiència de sortida. Alguns exemples són:


Per facilitar la reutilització de les frases i garantir coherència, hem creat funcions, les quals s'encarreguen de gestionar els missatges de manera modular i permeten que el sistema seleccioni aleatòriament les frases de cada conjunt en funció del moment de la conversa. Això fa que el sistema sigui més flexible i variat.

També hem afegit un petit efecte d’escriptura als missatges, com si el sistema "escrivís" el text abans de mostrar-lo. Això simula millor una conversa real i dona la sensació que l’assistent està pensant abans de respondre, millorar així l'experiència d'interacció.

Una altra millora important és la gestió de les respostes fora de rang. Quan l'usuari introdueix una resposta que no està dins dels valors esperats, el sistema li torna a preguntar de manera clara i amable. Per exemple, si l'usuari ha d'introduir un número i posa un valor incorrecte (com una cadena de text o un número fora del rang permès), el sistema li recorda que la resposta ha de ser un número sencer dins del rang especificat, i li torna a fer la pregunta fins que obtingui una resposta vàlida. Aquest enfocament ajuda a evitar frustracions i millora l'experiència d'ús.

A més d’aquestes millores, hem afegit una capa de personalització en la qual l'usuari pot modificar les seves preferències just abans de que el servei et proprocioni una recomanació segons les preferències indicades. Així, si vol canviar una secció concreta de les seves preferències (com el tipus d'habitatge, el pressupost o la ubicació), el sistema li permet fer-ho de manera senzilla i ràpida, sense necessitat de tornar a passar per totes les preguntes inicials. Aquesta flexibilitat afegeix una altra capa de comoditat a l'usuari, que pot adaptar la cerca segons les seves necessitats, fins i tot durant la conversa.

In [ ]:
import time, random, re, unicodedata

SALUTACIONS = [
    "Benvingut/da! Sóc el teu assistent per trobar casa.",
    "Hola! Comencem a perfilar l'habitatge ideal.",
    "Bones! Encantat d’ajudar-te a buscar pis o casa."
]

CONFIRMACIONS_BASE = [
    "Perfecte, gràcies!",
    "Genial, ho tinc apuntat.",
    "D'acord, anotat!",
    "Entesos!"
]

REPETIR_PREGUNTA_BASE = [
    "Em dones un número sencer? Per exemple, 2.",
    "Ho pots escriure amb un número senzill? (p. ex. 75)",
    "Uf, sembla que no ho he entès bé. Ho reformules amb un número?"
]

TRANSICIONS = [
    "Continuem.",
    "Seguim amb la següent pregunta…",
    "Anem avançant!"
]

COMIATS = [
    "Gràcies per fer-me servir. Que vagi molt bé!",
    "Ha estat un plaer ajudar-te. Fins aviat!",
    "Si necessites res més, aquí em tens. Adeu!"
]

def say(text: str):
    """
    Imprimeix el text a la pantalla com a resposta de l'assistent.
    """
    print(text)

def confirm(msg: str | None = None):
    """
    Confirma amb un missatge aleatori o el missatge personalitzat proporcionat.
    """
    if msg:
        say(msg)
    else:
        say(random.choice(CONFIRMACIONS_BASE))

def normalize(text: str) -> str:
    """
    Normalitza el text, eliminant espais extra i signes diacrítics (accents).
    """
    if text is None:
        return ""

    t = text.strip().lower()
    t = "".join(c for c in unicodedata.normalize("NFD", t) if unicodedata.category(c) != "Mn")
    return re.sub(r"\s+", " ", t)

def strip_token(s: str) -> str:
    """
    Elimina els caràcters no alfanumèrics del text normalitzat.
    """
    return re.sub(r"[^a-z0-9]+", "", normalize(s))

SKIP_WORDS = {"any", "igual", "cap", "sensepreferencia", "none", "skip"}
def is_skip(ans: str) -> bool:
    """
    Comprova si la resposta de l'usuari ha de ser saltada.
    """
    return strip_token(ans) in SKIP_WORDS

def to_bool(s):
    """
    Converteix una cadena a un valor booleà (True/False) basat en paraules clau com 'sí', 'no', 'true', 'false'.
    """
    if s is None:
        return None

    t = strip_token(s)
    T = {"si","s","yes","y","true","1","ok","vale","clar","correcte","sure"}
    F = {"no","n","false","0","never","mai"}

    if t in T:
        return True
    if t in F:
        return False
    return None

def find_first_int(s: str):
    """
    Busca el primer número sencer en una cadena i el retorna com a enter.
    """
    if s is None:
        return None

    m = re.search(r"[-+]?\d+", str(s))
    return int(m.group()) if m else None

def eco_prob(p=0.65):
    """
    Retorna un valor boolean segons una probabilitat aleatòria (per simular eco de respostes).
    """
    return random.random() < p

def format_echo(label, val, unit=None):
    """
    Formateja el missatge d'eco per a un valor determinat, afegint la unitat si cal.
    """
    if unit:
        return f"Apuntat! {val} {unit} per a {label}."

    if label:
        return f"Perfecte, anoto {val} per a {label}."

    return f"Anotat, {val}."

def ask_text(prompt: str, allow_skip: bool = True):
    """
    Fes una pregunta de text a l'usuari i permet saltar la pregunta amb 'any'.
    """
    print(prompt)
    ans = input("» Usuari: ").strip()

    if normalize(ans) == "quit":
        raise SystemExit

    if allow_skip and is_skip(ans):
        say("Cap problema, ho salto.")
        return None

    return ans

def ask_yesno(prompt: str, allow_skip: bool = True, label: str | None = None):
    """
    Pregunta a l'usuari una resposta afirmativa o negativa i permet saltar amb 'any'.
    """
    while True:
        ans = ask_text(f"{prompt} (sí/no" + (" o 'any' per saltar" if allow_skip else "") + "):", allow_skip)
        if ans is None:
            return None

        b = to_bool(ans)
        if b is not None:
            if eco_prob(0.55) and label:
                confirm(f"Perfecte, {'sí' if b else 'no'} a {label}. Ho apunto.")
            else:
                confirm()
            return b

        say(random.choice(REPETIR_PREGUNTA_BASE))

def ask_int(prompt: str, allow_skip: bool = True, minv: int | None = None,
            maxv: int | None = None, allow_negative: bool = False,
            label: str | None = None, unit: str | None = None):
    """
    Fes una pregunta de tipus enter a l'usuari, permetent opcions per a saltar-la, validar el rang i altres condicions.
    """
    while True:
        ans = ask_text(f"{prompt}" + ("" if not allow_skip else " (o 'any' per saltar)"), allow_skip)
        if ans is None:
            return None

        val = find_first_int(ans)
        if val is None:
            say("Uf, no he sabut llegir cap número. Si em poses un enter (per ex. 3), ho clavo!")
            continue

        if not allow_negative and val < 0:
            label_txt = label or "aquest camp"
            if minv is not None and maxv is not None:
                say(f"Ho sento, però no és possible tenir valors negatius de {label_txt}. "
                    f"Necessitaria que em donessis un valor entre {minv} i {maxv}; o escriu «any» per saltar.")
            else:
                say(f"Ho sento, però no és possible tenir valors negatius de {label_txt}. "
                    f"Pots escriure «any» per saltar.")
            continue

        if (minv is not None and val < minv) or (maxv is not None and val > maxv):
            label_txt = label or "aquest camp"
            if minv is not None and maxv is not None:
                say(f"Per {label_txt}, necessitaria un valor entre {minv} i {maxv}; o escriu «any» per saltar.")
            elif minv is not None:
                say(f"Per {label_txt}, hauria de ser com a mínim {minv}; o escriu «any» per saltar.")
            else:
                say(f"Per {label_txt}, hauria de ser com a màxim {maxv}; o escriu «any» per saltar.")
            continue

        confirm(format_echo(label, val, unit) if eco_prob(0.7) else None)
        return val

def ask_location_from_options(options: list[str]):
    """
    Pregunta a l'usuari per una ubicació de les opcions disponibles.
    """
    if not options:
        return None
    norm_map = {normalize(opt): opt for opt in options if opt}
    say("\nOn t'agradaria viure? Opcions: " + ", ".join(options))
    while True:
        loc = ask_text("Escriu una de les opcions exactes (o 'any' per saltar):")
        if loc is None:
            return None
        nloc = normalize(loc)
        if nloc in norm_map:
            chosen = norm_map[nloc]
            if eco_prob(0.6):
                say(f"Perfecte, ubicació «{chosen}» anotada.")
            return chosen
        say("No la tinc a la llista. Si vols, escriu 'any' per saltar; si no, prova una de les opcions mostrades.")

EDITABLE_SECTIONS_HELP = (
    "Pots escriure una d’aquestes seccions: "
    "habitacions, banys, pressupost, metres, ubicació, tipus, ingressos, planta, terrassa, ascensor, comercial.\n"
    "O bé escriu 'any' per saltar."
)

def summarize_prefs(prefs: dict):
    """
    Resumeix les preferències de l'usuari abans de continuar.
    """
    say("\nAbans de continuar, et faig un petit resum del que m’has comentat:")
    time.sleep(1)
    if not prefs:
        say("No has especificat cap preferència en concret, així que et mostraré opcions generals.")
        return
    for k, v in prefs.items():
        if k == "bedrooms":
            say(f"  Has dit que vols {v} habitacio/ns.")
            time.sleep(0.5)
        elif k == "bathrooms":
            say(f"  I que necessites {v} bany/s.")
            time.sleep(0.5)
        elif k == "price":
            say(f"  El teu pressupost aproximat és de {v}.")
            time.sleep(0.5)
        elif k == "square_meters":
            say(f"  Busques uns {v} metres quadrats.")
            time.sleep(0.5)
        elif k == "location":
            say(f"  La ubicació que prefereixes és «{v}».")
            time.sleep(0.5)
        elif k == "type":
            say(f"  T’interessa habitatge per {v}.")
            time.sleep(0.5)
        elif k == "income_monthly":
            say(f"  Has indicat uns ingressos mensuals de {v} €.")
            time.sleep(0.5)
        elif k == "min_floor":
            say(f"  Vols viure a partir de la planta {v}.")
            time.sleep(0.5)
        elif k == "want_terrace":
            say("  T’agradaria que tingués terrassa." if v else "  Has dit que no és imprescindible tenir terrassa.")
            time.sleep(0.5)
        elif k == "want_elevator":
            say("  Vols que hi hagi ascensor." if v else "  Has dit que no cal ascensor.")
            time.sleep(0.5)
        elif k == "want_commercial_use":
            say("  El vols utilitzar per a ús comercial." if v else "  No el destinaràs a ús comercial.")
            time.sleep(0.5)
        else:
            say(f"• {k}: {v}")

def _section_key_from_user(text: str) -> str | None:
    """
    Converteix el text a una clau per a les preferències de l'usuari.
    """
    t = normalize(text)
    aliases = {
        "bedrooms": {"habitacions", "hab", "rooms", "room", "habitacion", "dormitoris"},
        "bathrooms": {"banys", "bany", "baths", "bath", "banos"},
        "price": {"pressupost", "preu", "price", "budget"},
        "square_meters": {"metres", "m2", "metres quadrats", "superficie", "metros"},
        "location": {"ubicacio", "ubicación", "ubicacion", "lloc", "localitzacio", "localizacion", "location"},
        "type": {"tipus", "tipo", "type", "compra", "lloguer", "sale", "rent"},
        "income_monthly": {"ingressos", "salari", "sou", "renda", "income"},
        "min_floor": {"planta", "pis", "altura", "floor"},
        "want_terrace": {"terrassa", "terraza", "terrace"},
        "want_elevator": {"ascensor", "elevator", "lift"},
        "want_commercial_use": {"comercial", "us comercial", "uso comercial", "commercial"},
    }
    for key, names in aliases.items():
        if t in names:
            return key
    return None

def modify_section_once(section_key: str, prefs: dict, data: dict):
    """
    Modifica les preferències de l'usuari per una secció concreta.
    """
    if section_key == "bedrooms":
        val = ask_int("Quantes habitacions necessites? (1–5)", minv=1, maxv=5, label="habitacions", unit="habitacions")
        if val is None: prefs.pop("bedrooms", None)
        else: prefs["bedrooms"] = str(val)

    elif section_key == "bathrooms":
        val = ask_int("Quants banys necessites? (1–3)", minv=1, maxv=3, label="banys", unit="banys")
        if val is None: prefs.pop("bathrooms", None)
        else: prefs["bathrooms"] = str(val)

    elif section_key == "price":
        val = ask_text("Quin és el teu pressupost? (ex. 700k o 1200 en lloguer)")
        if val is None: prefs.pop("price", None)
        else: prefs["price"] = val

    elif section_key == "square_meters":
        val = ask_int("Quants metres quadrats necessites? (30–200)", minv=30, maxv=200, label="metres quadrats", unit="m²")
        if val is None: prefs.pop("square_meters", None)
        else: prefs["square_meters"] = str(val)

    elif section_key == "location":
        options = sorted({(h.get("location") or "").strip() for h in data.get("houses", []) if h.get("location")})
        if options:
            chosen = ask_location_from_options(options)
            if chosen is None: prefs.pop("location", None)
            else: prefs["location"] = chosen

    elif section_key == "type":
        while True:
            t = ask_text("Vols comprar o llogar? (sale/rent, o 'any' per saltar):")
            if t is None:
                prefs.pop("type", None)
                break
            t_norm = normalize(t)
            if t_norm in {"sale", "rent"}:
                prefs["type"] = t_norm
                confirm("D'acord, anotat.")
                break
            say("Per fer-ho bé, indica «sale» o «rent». O escriu «any» per no filtrar.")

    elif section_key == "income_monthly":
        val = ask_int("Ingressos mensuals de la llar (€)", allow_skip=True, minv=0, label="ingressos mensuals", unit="€")
        if val is None: prefs.pop("income_monthly", None)
        else: prefs["income_monthly"] = val

    elif section_key == "min_floor":
        val = ask_int("Planta mínima on vols viure? (0 = PB)", allow_skip=True, minv=0, label="planta mínima")
        if val is None: prefs.pop("min_floor", None)
        else: prefs["min_floor"] = val

    elif section_key == "want_terrace":
        val = ask_yesno("Vols terrassa?", allow_skip=True, label="terrassa")
        if val is None: prefs.pop("want_terrace", None)
        else: prefs["want_terrace"] = val

    elif section_key == "want_elevator":
        val = ask_yesno("Vols ascensor?", allow_skip=True, label="ascensor")
        if val is None: prefs.pop("want_elevator", None)
        else: prefs["want_elevator"] = val

    elif section_key == "want_commercial_use":
        val = ask_yesno("Faràs ús comercial (negoci)?", allow_skip=True, label="ús comercial")
        if val is None: prefs.pop("want_commercial_use", None)
        else: prefs["want_commercial_use"] = val

def edit_preferences_loop(prefs: dict, data: dict):
    """
    Permet modificar les preferències de l'usuari fins que confirmi que tot està correcte.
    """
    while True:
        ok = ask_yesno("\nEstà tot bé tal com ho tenim?", allow_skip=False, label="confirmació")
        if ok:
            confirm("Perfecte, seguim endavant.")
            return

        say("\nCap problema. Quina secció vols canviar?")
        say(EDITABLE_SECTIONS_HELP)

        sec = ask_text("Escriu el nom de la secció (o 'any' per no canviar res):")
        if sec is None:
            confirm("Entesos, no toco res.")
            continue

        while True:
            key = _section_key_from_user(sec)
            if key is None:
                say("No he identificat aquesta secció. Prova amb algun dels noms de la llista.")
                sec = ask_text("Escriu el nom de la secció (o 'any' per no canviar res):")
                if sec is None:
                    say("Entesos, no canvio res.")
                    break
                continue

            modify_section_once(key, prefs, data)
            summarize_prefs(prefs)
            break

def run_dialogue(data):
    """
    Controla el diàleg amb l'usuari per recollir preferències i buscar cases que s'ajustin a aquestes.
    """
    say(random.choice(SALUTACIONS))
    time.sleep(1)
    say("\nPots escriure 'quit' per sortir o 'any' per saltar una preferència.\n")
    time.sleep(1)
    say("Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.")
    time.sleep(1)
    say("Comencem!")
    time.sleep(1)

    prefs = {}

    val = ask_int("\nQuantes habitacions necessites? (1–5)", minv=1, maxv=5, label="habitacions", unit="habitacions")
    if val is not None: prefs["bedrooms"] = str(val)

    val = ask_int("\nQuants banys necessites? (1–3)", minv=1, maxv=3, label="banys", unit="banys")
    if val is not None: prefs["bathrooms"] = str(val)

    budget = ask_text("\nQuin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)")
    if budget is not None:
        prefs["price"] = budget
        if eco_prob(0.5):
            say(f"Entesos, treballaré amb el pressupost «{budget}».")

    sqm = ask_int("\nQuants metres quadrats necessites? (30–200)", minv=30, maxv=200, label="metres quadrats", unit="m²")
    if sqm is not None: prefs["square_meters"] = str(sqm)

    options = sorted({(h.get("location") or "").strip() for h in data.get("houses", []) if h.get("location")})
    if options:
        chosen_loc = ask_location_from_options(options)
        if chosen_loc is not None:
            prefs["location"] = chosen_loc
    say(random.choice(TRANSICIONS))

    while True:
        t = ask_text("\nVols comprar o llogar? (sale/rent, o 'any' per saltar):")
        if t is None: break
        t_norm = normalize(t)
        if t_norm in {"sale","rent"}:
            prefs["type"] = t_norm
            confirm(f"Apuntat: {t_norm}.")
            break
        say("Per fer-ho bé, indica «sale» o «rent». Si ho prefereixes, pots escriure «any» i no ho filtraré.")

    if prefs.get("type") == "rent":
        inc = ask_int("\nIngressos mensuals de la llar (€)", allow_skip=True, minv=0, label="ingressos mensuals", unit="€")
        if inc is not None: prefs["income_monthly"] = inc

    mf = ask_int("\nPlanta mínima on vols viure? (0 = PB)", allow_skip=True, minv=0, label="planta mínima")
    if mf is not None: prefs["min_floor"] = mf

    terr = ask_yesno("\nVols terrassa?", allow_skip=True, label="terrassa")
    if terr is not None: prefs["want_terrace"] = terr

    elev = ask_yesno("\nVols ascensor?", allow_skip=True, label="ascensor")
    if elev is not None: prefs["want_elevator"] = elev

    comm = ask_yesno("\nFaràs ús comercial (negoci)?", allow_skip=True, label="ús comercial")
    if comm is not None: prefs["want_commercial_use"] = comm

    summarize_prefs(prefs)
    edit_preferences_loop(prefs, data)

    suitable = find_suitable_houses(data, prefs)
    if suitable:
        say("\nSuper! He trobat opcions que s'hi ajusten:\n")
        for h in suitable:
            price = h.get('price')
            desc = (
                f"ID {h.get('id')} — {str(h.get('type') or '').upper()} — "
                f"{h.get('bedrooms')} hab / {h.get('bathrooms')} bany — "
                f"{h.get('square_meters')} m² — Planta {h.get('floor')} — "
                f"{'Terrassa' if str(h.get('terrace')).lower() in ['yes','sí','si','true','1'] else 'Sense terrassa'} — "
                f"{'Ascensor' if str(h.get('elevator')).lower() in ['yes','sí','si','true','1'] else 'Sense ascensor'} — "
                f"{h.get('location')} — Preu: {price}"
            )
            say("• " + desc)
    else:
        say("\nNo he trobat opcions amb aquests criteris.")
        say("\nSi vols, podem relaxar alguna condició (p.ex. pujar pressupost o baixar m²) i ho tornem a provar.")

    say("\n" + random.choice(COMIATS))


In [ ]:
try:
    run_dialogue(data)
except SystemExit:
    say("\nHe aturat la conversa tal com m'has demanat.")


Bones! Encantat d’ajudar-te a buscar pis o casa.

Pots escriure 'quit' per sortir o 'any' per saltar una preferència.

Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.
Comencem!

Quantes habitacions necessites? (1–5) (o 'any' per saltar)
» Usuari: 3
Apuntat! 3 habitacions per a habitacions.

Quants banys necessites? (1–3) (o 'any' per saltar)
» Usuari: 2
Apuntat! 2 banys per a banys.

Quin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)
» Usuari: 1200
Entesos, treballaré amb el pressupost «1200».

Quants metres quadrats necessites? (30–200) (o 'any' per saltar)
» Usuari: 60 
Apuntat! 60 m² per a metres quadrats.

On t'agradaria viure? Opcions: Badalona, Barcelona, Esplugues de Llobregat, L'Hospitalet de Llobregat, Santa Coloma de Gramenet
Escriu la ubicació (pots fer faltes; o 'any' per saltar):
» Usuari: santa coloma de gram 
Volies dir «Santa Coloma de Gramenet»? (sí/no):
» Usuari: si 
Perfecte, sí a confirmació ubicació. Ho ap

En aquesta primera millora hem afegit una capa d’humanització al sistema per fer que la conversa sigui més natural i propera. Per aconseguir-ho hem definit conjunts de frases en català per a diferents moments de la interacció (SALUTACIONS, CONFIRMACIONS, TRANSICIONS, COMIATS) i hem creat funcions específiques (dir_missatge, dir_confirmacio, etc.) que gestionen la sortida de manera variada i amable. Això ha permès que el sistema confirmi respostes, faci transicions suaus entre preguntes i acabi amb un comiat proper, millorant l’experiència d’usuari.

Durant les execucions, hem detectat i corregit diversos problemes que afectaven l'experiència de l'usuari. Un dels errors més destacats va ser que el sistema confonia la resposta “no” amb una ordre per saltar la pregunta, cosa que feia perdre informació.
Per solucionar aquests errors, vam ajustar la llista de paraules reservades per saltar preguntes i vam afegir una neteja de puntuació abans de processar les respostes. Així, podem identificar millor respostes com "sí" o "no", independentment de la puntuació que les acompanyi. A més a més, si el programa no enetn alguna de les respostes prorpcionades, li diuq ue per tal de fer-ho bé ha d'ndicar la resposta amb el format corresponent a cada una de les preguntes.

A més, vam identificar que el sistema acceptava valors fora del rang per a preguntes numèriques, cosa que podia portar a respostes incorrectes o inesperades. Per corregir-ho, hem afegit un control que fa que el sistema repeteixi la pregunta si la resposta no compleix amb els valors esperats. Aquesta mesura ajuda a evitar errors i assegura que l'usuari introdueixi respostes vàlides, millorant la qualitat del diàleg.
Finalment, també hem adaptat els missatges d'error perquè siguin més coherents amb el tipus de pregunta (sí/no o numèrica). Això fa que els missatges siguin més clars i adequats al context de la conversa, millorant la comprensió de l'usuari.

Amb aquests ajustos, el sistema funciona de manera més fluida i tolerant, i l’usuari percep un diàleg més humà i comprensible. Hem aconseguit que el sistema sigui més fiable en la gestió de respostes incorrectes i que el flux de la conversa sigui més coherent i natural.

---

**Millora 2: Ubicació tolerant a faltes (coincidència aproximada) i guia del 35% en lloguer**

L’objectiu d’aquesta millora és reduir la fricció a l’hora d’introduir preferències i evitar falsos negatius per detalls de format. En concret, hem afegit:
*   **MILLORA 2.1)**
**Coincidència aproximada (fuzzy matching) per a la ubicació:**
Sovint els usuaris escriuen el barri o la ciutat amb variacions (“barcelona”, “BARCELONA”, “barcelon”, “barna”…).
Amb la funció tria_ubicacio_semblant, el sistema compara l’entrada amb la llista d’ubicacions disponibles i, si la semblança supera un llindar (0.75 per defecte), la mapa automàticament a l’opció correcta (p. ex., “Barcelona”).
Això millora la robustesa del diàleg i redueix repreguntes innecessàries.
*   **MILLORA 2.2)**
**Missatge d’ajuda sobre el 35% dels ingressos en lloguer:**
Quan l’usuari selecciona el tipus rent, el sistema mostra un recordatori: “No destinar més del 35% dels ingressos al lloguer.”
Aquesta pista ofereix orientació pràctica i prepara el terreny per a futures millores (p. ex., calcular automàticament el límit recomanat i oferir-lo com a pressupost mensual).


Ambdós canvis no alteren la lògica de filtratge existent (continuem utilitzant find_suitable_houses), però fan que el sistema sigui més tolerant, intel·ligent i amable en la interacció, millorant l’experiència d’usuari sense comprometre el control ni la transparència.

**MILLORA 2.1**

In [ ]:
import difflib

def _best_location_match(entrada: str, options: list[str]) -> tuple[str | None, float]:
    """
    Troba la millor coincidència entre l'entrada de l'usuari i les opcions disponibles basant-se en la similitud de cadenes.
    """
    if not entrada or not options:
        return (None, 0.0)

    # Crear un diccionari de les opcions normalitzades
    norm_map = {normalize(o): o for o in options if o}
    entrada_n = normalize(entrada)

    # Crear una llista de candidats amb la similitud de cadenes
    candidates = []
    for nopt, orig in norm_map.items():
        score = difflib.SequenceMatcher(None, entrada_n, nopt).ratio()
        candidates.append((orig, score))

    # Ordenar per la millor coincidència (màxim puntatge)
    candidates.sort(key=lambda x: x[1], reverse=True)

    # Retornar la millor coincidència o None si no hi ha candidats
    return candidates[0] if candidates else (None, 0.0)


def ask_location_from_options(options: list[str], llindar: float = 0.75):
    """
    Demana a l'usuari una ubicació i comprova la similitud amb les opcions disponibles
    """
    if not options:
        return None

    # Mostrar les opcions disponibles
    say("\nOn t'agradaria viure? Opcions: " + ", ".join(options))

    # Crear el diccionari de les opcions normalitzades
    norm_map = {normalize(o): o for o in options}

    while True:
        # Preguntar per la ubicació
        loc_in = ask_text("Escriu la ubicació (pots fer faltes; o 'any' per saltar):")
        if loc_in is None:
            return None

        # Normalitzar la resposta de l'usuari
        loc_n = normalize(loc_in)

        # Si la ubicació és exactament una opció, la seleccionem
        if loc_n in norm_map:
            chosen = norm_map[loc_n]
            if eco_prob(0.6):
                say(f"Perfecte, ubicació «{chosen}» anotada.")
            return chosen

        # Si no trobem una coincidència exacta, busquem la millor coincidència
        sugg, score = _best_location_match(loc_in, options)

        # Si la coincidència és prou bona, oferim una confirmació
        if sugg and score >= llindar:
            ok = ask_yesno(f"Volies dir «{sugg}»?", allow_skip=False, label="confirmació ubicació")
            if ok:
                confirm(f"Ubicació «{sugg}» anotada.")
                return sugg
            else:
                say("Cap problema! Torna a escriure la ubicació (o 'any' per saltar).")
                continue

        # Si no hi ha una coincidència prou bona, demanem que escrigui una opció vàlida
        say("No he trobat una ubicació prou semblant. Prova una de les opcions de la llista (o 'any' per saltar).")


In [ ]:
def run_dialogue(data):
    say(random.choice(SALUTACIONS))
    time.sleep(1)
    say("\nPots escriure 'quit' per sortir o 'any' per saltar una preferència.\n")
    time.sleep(1)
    say("Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.")
    time.sleep(1)
    say("Comencem!")
    time.sleep(1)

    prefs = {}

    val = ask_int("\nQuantes habitacions necessites? (1–5)", minv=1, maxv=5, label="habitacions", unit="habitacions")
    if val is not None: prefs["bedrooms"] = str(val)

    val = ask_int("\nQuants banys necessites? (1–3)", minv=1, maxv=3, label="banys", unit="banys")
    if val is not None: prefs["bathrooms"] = str(val)

    budget = ask_text("\nQuin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)")
    if budget is not None:
        prefs["price"] = budget
        if eco_prob(0.5):
            say(f"Entesos, treballaré amb el pressupost «{budget}».")

    sqm = ask_int("\nQuants metres quadrats necessites? (30–200)", minv=30, maxv=200, label="metres quadrats", unit="m²")
    if sqm is not None: prefs["square_meters"] = str(sqm)

    options = sorted({(h.get("location") or "").strip() for h in data.get("houses", []) if h.get("location")})
    if options:
        chosen_loc = ask_location_from_options(options, llindar=0.75)
        if chosen_loc is not None:
            prefs["location"] = chosen_loc

    say(random.choice(TRANSICIONS))

    while True:
        t = ask_text("\nVols comprar o llogar? (sale/rent, o 'any' per saltar):")
        if t is None: break
        t_norm = normalize(t)
        if t_norm in {"sale","rent"}:
            prefs["type"] = t_norm
            confirm(f"Apuntat: {t_norm}.")
            break
        say("Per fer-ho bé, indica «sale» o «rent». Si ho prefereixes, pots escriure «any» i no ho filtraré.")

    if prefs.get("type") == "rent":
        inc = ask_int("\nIngressos mensuals de la llar (€)", allow_skip=True, minv=0, label="ingressos mensuals", unit="€")
        if inc is not None: prefs["income_monthly"] = inc

    mf = ask_int("\nPlanta mínima on vols viure? (0 = PB)", allow_skip=True, minv=0, label="planta mínima")
    if mf is not None: prefs["min_floor"] = mf

    terr = ask_yesno("\nVols terrassa?", allow_skip=True, label="terrassa")
    if terr is not None: prefs["want_terrace"] = terr

    elev = ask_yesno("\nVols ascensor?", allow_skip=True, label="ascensor")
    if elev is not None: prefs["want_elevator"] = elev

    comm = ask_yesno("\nFaràs ús comercial (negoci)?", allow_skip=True, label="ús comercial")
    if comm is not None: prefs["want_commercial_use"] = comm

    summarize_prefs(prefs)
    edit_preferences_loop(prefs, data)

    suitable = find_suitable_houses(data, prefs)
    if suitable:
        say("\nSuper! He trobat opcions que s'hi ajusten:\n")
        for h in suitable:
            price = h.get('price')
            desc = (
                f"ID {h.get('id')} — {str(h.get('type') or '').upper()} — "
                f"{h.get('bedrooms')} hab / {h.get('bathrooms')} bany — "
                f"{h.get('square_meters')} m² — Planta {h.get('floor')} — "
                f"{'Terrassa' if str(h.get('terrace')).lower() in ['yes','sí','si','true','1'] else 'Sense terrassa'} — "
                f"{'Ascensor' if str(h.get('elevator')).lower() in ['yes','sí','si','true','1'] else 'Sense ascensor'} — "
                f"{h.get('location')} — Preu: {price}"
            )
            say("• " + desc)
    else:
        say("\nNo he trobat opcions amb aquests criteris.")
        say("Si vols, podem relaxar alguna condició (p.ex. pujar pressupost o baixar m²) i ho tornem a provar.")

    say("\n" + random.choice(COMIATS))

In [ ]:
try:
    run_dialogue(data)
except SystemExit:
    say("\nHe aturat la conversa tal com m'has demanat.")

Hola! Comencem a perfilar l'habitatge ideal.

Pots escriure 'quit' per sortir o 'any' per saltar una preferència.

Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.
Comencem!

Quantes habitacions necessites? (1–5) (o 'any' per saltar)
» Usuari: any 
Cap problema, ho salto.

Quants banys necessites? (1–3) (o 'any' per saltar)
» Usuari: 2
Entesos!

Quin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)
» Usuari: 1200 
Entesos, treballaré amb el pressupost «1200».

Quants metres quadrats necessites? (30–200) (o 'any' per saltar)
» Usuari: 250
Per metres quadrats, necessitaria un valor entre 30 i 200; o escriu «any» per saltar.

Quants metres quadrats necessites? (30–200) (o 'any' per saltar)
» Usuari: quit

He aturat la conversa tal com m'has demanat.


**MILLORA 2.2**

In [ ]:
def run_dialogue(data):
    say(random.choice(SALUTACIONS))
    time.sleep(1)
    say("\nPots escriure 'quit' per sortir o 'any' per saltar una preferència.\n")
    time.sleep(1)
    say("Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.")
    time.sleep(1)
    say("Comencem!")
    time.sleep(1)

    prefs = {}

    val = ask_int("\nQuantes habitacions necessites? (1–5)", minv=1, maxv=5, label="habitacions", unit="habitacions")
    if val is not None: prefs["bedrooms"] = str(val)

    val = ask_int("\nQuants banys necessites? (1–3)", minv=1, maxv=3, label="banys", unit="banys")
    if val is not None: prefs["bathrooms"] = str(val)

    budget = ask_text("\nQuin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)")
    if budget is not None:
        prefs["price"] = budget
        if eco_prob(0.5):
            say(f"Entesos, treballaré amb el pressupost «{budget}».")

    sqm = ask_int("\nQuants metres quadrats necessites? (30–200)", minv=30, maxv=200, label="metres quadrats", unit="m²")
    if sqm is not None: prefs["square_meters"] = str(sqm)

    # --- UBICACIÓ amb fuzzy integrat a ask_location_from_options ---
    options = sorted({(h.get("location") or "").strip() for h in data.get("houses", []) if h.get("location")})
    if options:
        chosen_loc = ask_location_from_options(options, llindar=0.75)
        if chosen_loc is not None:
            prefs["location"] = chosen_loc
    # --- fi ubicació ---
    say(random.choice(TRANSICIONS))

    while True:
        t = ask_text("\nVols comprar o llogar? (sale/rent, o 'any' per saltar):")
        if t is None:
            break
        t_norm = normalize(t)
        if t_norm in {"sale","rent"}:
            prefs["type"] = t_norm
            confirm(f"Apuntat: {t_norm}.")
            break
        say("Per fer-ho bé, indica «sale» o «rent». Si ho prefereixes, pots escriure «any» i no ho filtraré.")

    if prefs.get("type") == "rent":
        say("     \nRecorda: es recomana no destinar més del 35% dels ingressos mensuals al lloguer.")
        inc = ask_int("Ingressos mensuals de la llar (€)", allow_skip=True, minv=0, label="ingressos mensuals", unit="€")
        if inc is not None:
            prefs["income_monthly"] = inc

    mf = ask_int("\nPlanta mínima on vols viure? (0 = PB)", allow_skip=True, minv=0, label="planta mínima")
    if mf is not None:
        prefs["min_floor"] = mf

    terr = ask_yesno("\nVols terrassa?", allow_skip=True, label="terrassa")
    if terr is not None:
        prefs["want_terrace"] = terr

    elev = ask_yesno("\nVols ascensor?", allow_skip=True, label="ascensor")
    if elev is not None:
        prefs["want_elevator"] = elev

    comm = ask_yesno("\nFaràs ús comercial (negoci)?", allow_skip=True, label="ús comercial")
    if comm is not None:
        prefs["want_commercial_use"] = comm

    summarize_prefs(prefs)
    edit_preferences_loop(prefs, data)

    suitable = find_suitable_houses(data, prefs)
    if suitable:
        say("\nSuper! He trobat opcions que s'hi ajusten:\n")
        for h in suitable:
            price = h.get('price')
            desc = (
                f"ID {h.get('id')} — {str(h.get('type') or '').upper()} — "
                f"{h.get('bedrooms')} hab / {h.get('bathrooms')} bany — "
                f"{h.get('square_meters')} m² — Planta {h.get('floor')} — "
                f"{'Terrassa' if str(h.get('terrace')).lower() in ['yes','sí','si','true','1'] else 'Sense terrassa'} — "
                f"{'Ascensor' if str(h.get('elevator')).lower() in ['yes','sí','si','true','1'] else 'Sense ascensor'} — "
                f"{h.get('location')} — Preu: {price}"
            )
            say("• " + desc)
    else:
        say("\nNo he trobat opcions amb aquests criteris.")
        say("Si vols, podem relaxar alguna condició (p.ex. pujar pressupost o baixar m²) i ho tornem a provar.")

    say("\n" + random.choice(COMIATS))

In [ ]:
try:
    run_dialogue(data)
except SystemExit:
    say("\nHe aturat la conversa tal com m'has demanat.")

Hola! Comencem a perfilar l'habitatge ideal.

Pots escriure 'quit' per sortir o 'any' per saltar una preferència.

Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.
Comencem!

Quantes habitacions necessites? (1–5) (o 'any' per saltar)
» Usuari: 3
Apuntat! 3 habitacions per a habitacions.

Quants banys necessites? (1–3) (o 'any' per saltar)
» Usuari: 2
Apuntat! 2 banys per a banys.

Quin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)
» Usuari: 1200

Quants metres quadrats necessites? (30–200) (o 'any' per saltar)
» Usuari: 60
Perfecte, gràcies!

On t'agradaria viure? Opcions: Badalona, Barcelona, Esplugues de Llobregat, L'Hospitalet de Llobregat, Santa Coloma de Gramenet
Escriu la ubicació (pots fer faltes; o 'any' per saltar):
» Usuari: santa Coloma de gram
Volies dir «Santa Coloma de Gramenet»? (sí/no):
» Usuari: si
Perfecte, sí a confirmació ubicació. Ho apunto.
Ubicació «Santa Coloma de Gramenet» anotada.
Continuem.

Vols comp

Amb aquesta millora, hem implementat un sistema que permet detectar millor les coincidències entre l'entrada de l'usuari i les opcions disponibles, fins i tot quan l'usuari fa faltes d'ortografia o escriu respostes parcialment correctes.

---

**MILLORA 3: Flexibilitat en les recomanacions d'habitatges**

En aquesta millora hem fet que el sistema no sigui tan rígid a l’hora de filtrar habitatges. Fins ara, només es mostraven aquells que complien exactament tots els criteris de l’usuari. Ara bé, a la realitat pot ser interessant veure pisos que, tot i no coincidir al 100% amb les preferències, podem ser una bona opció a tenir en compte.

Així, el sistema és capaç de recomanar habitatges que tenen un preu igual o inferior al límit marcat, però que a la vegada aporten característiques superiors a les mínimes demanades (per exemple, més metres quadrats, més habitacions o més banys). D’aquesta manera, l’usuari pot descobrir opcions que potser no havia previst però que resulten atractives i, en molts casos, avantatjoses.

Aquesta millora fa que el recomanador sigui més flexible i realista, apropant-se a com ho faria un assessor humà: no només mostrar allò que encaixa estrictament amb els criteris, sinó també alternatives que representen una bona oportunitat de mercat.

In [ ]:
def print_three_buckets(strict_list, value_list, near_list, limit_per_bucket=5):
    """
    Mostra les opcions de les tres categories de recomanacions:
    1. Opcions estrictes (compleixen tots els criteris),
    2. Bones ofertes (preu igual o inferior al límit i més atributs),
    3. Alternatives properes (lleugerament per sota dels mínims, amb tolerància).
    """
    def yesno(txt):
        return "Sí" if str(txt).strip().lower() in {"yes", "sí", "si", "true", "1"} else "No"

    # Mostrar opcions estrictes (que compleixen tots els criteris)
    if strict_list:
        print("\nOpcions que S'ADAPTEN als teus criteris:\n")
        for h in strict_list[:limit_per_bucket]:
            print(f"ID {h.get('id')} — {str(h.get('type')).upper()} — "
                  f"{h.get('bedrooms')} hab / {h.get('bathrooms')} bany — "
                  f"{h.get('square_meters')} m² — Planta {h.get('floor')} — "
                  f"{'Terrassa' if yesno(h.get('terrace')) == 'Sí' else 'Sense terrassa'} — "
                  f"{'Ascensor' if yesno(h.get('elevator')) == 'Sí' else 'Sense ascensor'} — "
                  f"{h.get('location')} — Preu: {h.get('price')}")
        print()

    # Mostrar bones ofertes (preu igual o inferior al límit i més valor)
    if value_list:
        print("Bones ofertes (mateix o menys preu, però amb més valor):\n")
        for h in value_list[:limit_per_bucket]:
            print(f"ID {h.get('id')} — {str(h.get('type')).upper()} — "
                  f"{h.get('bedrooms')} hab / {h.get('bathrooms')} bany — "
                  f"{h.get('square_meters')} m² — Planta {h.get('floor')} — "
                  f"{h.get('location')} — Preu: {h.get('price')}")
        print()

    # Mostrar alternatives properes (lleugerament per sota dels mínims)
    if near_list:
        print("Alternatives properes (lleugerament per sota d'algun mínim):\n")
        for h in near_list[:limit_per_bucket]:
            print(f"ID {h.get('id')} — {str(h.get('type')).upper()} — "
                  f"{h.get('bedrooms')} hab / {h.get('bathrooms')} bany — "
                  f"{h.get('square_meters')} m² — Planta {h.get('floor')} — "
                  f"{h.get('location')} — Preu: {h.get('price')}")
        print()

    # Si no es troben opcions, informar a l'usuari
    if not strict_list and not value_list and not near_list:
        print("\nNo he trobat opcions amb aquests criteris.\n")

def find_value_houses(data, user_preferences):
    """
    Retorna habitatges que:
      - compleixen filtres durs (tipus, ubicació, límit preu/ingressos, exigències explícites),
      - i són 'bones ofertes': preu ≤ límit i algun atribut per SOBRE del mínim demanat
        (m², habitacions o banys).
    """
    houses = data.get('houses', [])
    if not houses:
        return []

    max_price   = _to_euros(user_preferences.get('price'))
    min_bed     = _to_int(user_preferences.get('bedrooms'))
    min_bath    = _to_int(user_preferences.get('bathrooms'))
    min_sqm     = _to_int(user_preferences.get('square_meters'))
    wanted_type = (user_preferences.get('type') or None)
    wanted_loc  = (user_preferences.get('location') or None)
    want_terr   = user_preferences.get('want_terrace')
    want_elev   = user_preferences.get('want_elevator')
    want_comm   = user_preferences.get('want_commercial_use')
    income      = user_preferences.get('income_monthly')

    results = []
    for h in houses:
        pr   = _to_euros(h.get('price'))
        bed  = _to_int(h.get('bedrooms'))
        bath = _to_int(h.get('bathrooms'))
        sqm  = _to_int(h.get('square_meters'))
        flr  = _to_int(h.get('floor'))
        typ  = (h.get('type') or '').strip().lower()
        loc  = (h.get('location') or '').strip().lower()
        terr = (str(h.get('terrace') or '').strip().lower() in {"yes","sí","si","true","1"})
        elev = (str(h.get('elevator') or '').strip().lower() in {"yes","sí","si","true","1"})
        comm = (str(h.get('commercial_use') or '').strip().lower() in {"yes","sí","si","true","1"})

        # --- filtres durs (no es relaxen) ---
        if wanted_type and typ != str(wanted_type).strip().lower():
            continue
        if wanted_loc and loc != str(wanted_loc).strip().lower():
            continue
        if wanted_type == "rent" and income is not None:
            max_aff = 0.35 * float(income)
            if pr is None or pr > max_aff:
                continue
        if want_terr is True and not terr:
            continue
        if want_elev is True and not elev:
            continue
        if want_comm is True and not comm:
            continue
        if max_price is not None and (pr is None or pr > max_price):
            # per ser 'millor oferta' exigim preu ≤ límit
            continue

        # --- valor afegit: per sobre del mínim demanat ---
        better_space = (min_sqm is not None and sqm is not None and sqm > min_sqm)
        better_bed   = (min_bed is not None and bed is not None and bed > min_bed)
        better_bath  = (min_bath is not None and bath is not None and bath > min_bath)

        # Si l'usuari no ha fixat cap mínim, no considerem 'millor oferta'
        any_min = any(v is not None for v in [min_sqm, min_bed, min_bath])

        if any_min and (better_space or better_bed or better_bath):
            results.append(h)

    # ordena per preu ascendent
    results.sort(key=lambda x: (_to_euros(x.get('price'))
                                if _to_euros(x.get('price')) is not None else float('inf')))
    return results


In [ ]:
def run_dialogue(data):
    """
    Controla el diàleg amb l'usuari per recollir preferències i buscar cases que s'ajustin a aquestes.
    """
    say(random.choice(SALUTACIONS))
    time.sleep(1)
    say("\nPots escriure 'quit' per sortir o 'any' per saltar una preferència.\n")
    time.sleep(1)
    say("Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.")
    time.sleep(1)
    say("Comencem!")
    time.sleep(1)

    prefs = {}

    # Recollir preferències de l'usuari
    val = ask_int("\nQuantes habitacions necessites? (1–5)", minv=1, maxv=5, label="habitacions", unit="habitacions")
    if val is not None: prefs["bedrooms"] = str(val)

    val = ask_int("\nQuants banys necessites? (1–3)", minv=1, maxv=3, label="banys", unit="banys")
    if val is not None: prefs["bathrooms"] = str(val)

    budget = ask_text("\nQuin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)")
    if budget is not None:
        prefs["price"] = budget
        if eco_prob(0.5):
            say(f"Entesos, treballaré amb el pressupost «{budget}».")

    sqm = ask_int("\nQuants metres quadrats necessites? (30–200)", minv=30, maxv=200, label="metres quadrats", unit="m²")
    if sqm is not None: prefs["square_meters"] = str(sqm)

    # UBICACIÓ amb fuzzy integrat a ask_location_from_options
    options = sorted({(h.get("location") or "").strip() for h in data.get("houses", []) if h.get("location")})
    if options:
        chosen_loc = ask_location_from_options(options, llindar=0.75)
        if chosen_loc is not None:
            prefs["location"] = chosen_loc

    say(random.choice(TRANSICIONS))

    # Tipus de propietat
    while True:
        t = ask_text("\nVols comprar o llogar? (sale/rent, o 'any' per saltar):")
        if t is None:
            break
        t_norm = normalize(t)
        if t_norm in {"sale", "rent"}:
            prefs["type"] = t_norm
            confirm(f"Apuntat: {t_norm}.")
            break
        say("Per fer-ho bé, indica «sale» o «rent». Si ho prefereixes, pots escriure «any» i no ho filtraré.")

    if prefs.get("type") == "rent":
        say("     \nRecorda: es recomana no destinar més del 35% dels ingressos mensuals al lloguer.")
        inc = ask_int("Ingressos mensuals de la llar (€)", allow_skip=True, minv=0, label="ingressos mensuals", unit="€")
        if inc is not None:
            prefs["income_monthly"] = inc

    # Planta mínima
    mf = ask_int("\nPlanta mínima on vols viure? (0 = PB)", allow_skip=True, minv=0, label="planta mínima")
    if mf is not None: prefs["min_floor"] = mf

    # Terrassa, Ascensor, Ús comercial
    terr = ask_yesno("\nVols terrassa?", allow_skip=True, label="terrassa")
    if terr is not None: prefs["want_terrace"] = terr

    elev = ask_yesno("\nVols ascensor?", allow_skip=True, label="ascensor")
    if elev is not None: prefs["want_elevator"] = elev

    comm = ask_yesno("\nFaràs ús comercial (negoci)?", allow_skip=True, label="ús comercial")
    if comm is not None: prefs["want_commercial_use"] = comm

    summarize_prefs(prefs)
    edit_preferences_loop(prefs, data)

    # Trobar cases adequades amb la nova flexibilitat de recomanacions
    strict, value, near = compute_recommendations(data, prefs)

    if strict or value or near:
        say("\nSuper! Aquí tens les meves recomanacions:\n")
        print_three_buckets(strict, value, near, limit_per_bucket=5)
    else:
        say("\nNo he trobat opcions amb aquests criteris.")
        say("Si vols, podem relaxar alguna condició (p.ex. pujar pressupost o baixar m²) i ho tornem a provar.")

    say("\n" + random.choice(COMIATS))


def compute_recommendations(data, prefs):
    """
    Retorna tres llistes:
      - strict: compleixen tots els criteris (amb relaxables passant sense penalització)
      - value: millors ofertes (preu ≤ límit i atributs per sobre del mínim)
      - near: near-misses (lleugerament per sota d'algun mínim, amb tolerància)
    """
    strict, near = find_suitable_houses_relaxed(data, prefs)
    value = find_value_houses(data, prefs)

    # Evitar duplicats: treu de 'value' els que ja són 'strict'
    strict_ids = {h.get('id') for h in strict if h.get('id') is not None}
    value = [h for h in value if h.get('id') not in strict_ids]

    return strict, value, near


def find_suitable_houses_relaxed(data, user_preferences, tol_sqm=0.15, tol_floor=1, tol_bed=1, tol_bath=1):
    houses = data.get('houses', [])
    if not houses:
        return [], []

    # Preferències
    min_bed = _to_int(user_preferences.get('bedrooms'))
    min_bath = _to_int(user_preferences.get('bathrooms'))
    min_sqm = _to_int(user_preferences.get('square_meters'))
    max_price = _to_euros(user_preferences.get('price'))
    wanted_loc = user_preferences.get('location')
    wanted_type = user_preferences.get('type')
    income = user_preferences.get('income_monthly')
    min_floor_pref = user_preferences.get('min_floor')
    want_terrace = user_preferences.get('want_terrace')
    want_elevator = user_preferences.get('want_elevator')
    want_commercial = user_preferences.get('want_commercial_use')

    strict, near = [], []

    for h in houses:
        bed = _to_int(h.get('bedrooms'))
        bath = _to_int(h.get('bathrooms'))
        sqm = _to_int(h.get('square_meters'))
        pr = _to_euros(h.get('price'))
        loc = (h.get('location') or '').strip().lower()
        typ = (h.get('type') or '').strip().lower()
        flr = _to_int(h.get('floor'))
        terr = (str(h.get('terrace') or '').strip().lower() in {"yes", "sí", "si", "true", "1"})
        elev = (str(h.get('elevator') or '').strip().lower() in {"yes", "sí", "si", "true", "1"})
        comm = (str(h.get('commercial_use') or '').strip().lower() in {"yes", "sí", "si", "true", "1"})

        # Filtres durs
        if wanted_type and typ != str(wanted_type).strip().lower():
            continue
        if wanted_loc and loc != str(wanted_loc).strip().lower():
            continue
        if wanted_type == "rent" and income is not None:
            if pr is None or pr > 0.35 * float(income):
                continue
        if max_price is not None and (pr is None or pr > max_price):
            continue
        if want_terrace is True and not terr:
            continue
        if want_elevator is True and not elev:
            continue
        if want_commercial is True and not comm:
            continue

        # Filtres relaxables
        fails = []

        if min_bed is not None:
            if bed < min_bed:
                diff = min_bed - bed
                if diff <= tol_bed:
                    fails.append(("bedrooms", diff))
                else:
                    continue

        if min_bath is not None:
            if bath < min_bath:
                diff = min_bath - bath
                if diff <= tol_bath:
                    fails.append(("bathrooms", diff))
                else:
                    continue

        if min_sqm is not None:
            if sqm < min_sqm:
                gap = (min_sqm - sqm) / float(min_sqm)
                if gap <= tol_sqm:
                    fails.append(("square_meters", gap))
                else:
                    continue

        if min_floor_pref is not None:
            if flr < int(min_floor_pref):
                diff = int(min_floor_pref) - flr
                if diff <= tol_floor:
                    fails.append(("floor", diff))
                else:
                    continue

        if not fails:
            strict.append(h)
        else:
            penalty = 0.0
            for k, d in fails:
                if d is None:
                    penalty += 1.0
                elif k == "square_meters":
                    penalty += d * 2.0
                else:
                    penalty += float(d)
            h["_near_penalty"] = penalty
            near.append(h)

    strict.sort(key=lambda x: (_to_euros(x.get('price')) if _to_euros(x.get('price')) is not None else float('inf')))
    near.sort(key=lambda x: (x.get("_near_penalty", 9e9), _to_euros(x.get('price')) if _to_euros(x.get('price')) is not None else float('inf')))
    return strict, near

In [47]:
try:
    run_dialogue(data)
except SystemExit:
    say("\nHe aturat la conversa tal com m'has demanat.")

Benvingut/da! Sóc el teu assistent per trobar casa.

Pots escriure 'quit' per sortir o 'any' per saltar una preferència.

Ara et faré unes preguntes que em permetran trobar aquesta casa que estàs buscant.
Comencem!

Quantes habitacions necessites? (1–5) (o 'any' per saltar)
» Usuari: any
Cap problema, ho salto.

Quants banys necessites? (1–3) (o 'any' per saltar)
» Usuari: any
Cap problema, ho salto.

Quin és el teu pressupost per a l'habitatge? (p.ex. 700k o 1200 en lloguer)
» Usuari: any
Cap problema, ho salto.

Quants metres quadrats necessites? (30–200) (o 'any' per saltar)
» Usuari: any
Cap problema, ho salto.

On t'agradaria viure? Opcions: Badalona, Barcelona, Esplugues de Llobregat, L'Hospitalet de Llobregat, Santa Coloma de Gramenet
Escriu la ubicació (pots fer faltes; o 'any' per saltar):
» Usuari: Barcelo
Volies dir «Barcelona»? (sí/no):
» Usuari: quir
Ho pots escriure amb un número senzill? (p. ex. 75)
Volies dir «Barcelona»? (sí/no):
» Usuari: quit

He aturat la conversa t

Amb la Millora 3, hem aconseguit fer que el sistema sigui més flexible i adaptable a les preferències de l'usuari, oferint recomanacions més variades. A més, la flexibilitat en els filtres permet suggerir opcions que, tot i no complir tots els criteris, poden ser bones alternatives, millorant així l'experiència d'usuari i augmentant les possibilitats de trobar una opció que s'ajusti a les seves necessitats.

----

<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Resultats (amb raonament).
3. Conversation flow del vostre sistema final.
4. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.


---